In [1]:
from theano import tensor as T
import lasagne_dev as nn
import numpy as np
import pickle
from input_process import read_images
import random
import math
import cmath
from g_gradient import *
import timeit
import unet
from collections import OrderedDict

In [ ]:
start = timeit.default_timer()
epoch = 1
shape = [5,1,116,116]

input_var = T.tensor4('input_var')  
label_var = T.tensor4('label_var')

[network, loss, test_loss, test_acc, output_det] = unet.network(input_var, label_var, shape)


params = nn.layers.get_all_params(network)
lr = theano.shared(nn.utils.floatX(1e-4)) # learning rate
updates = nn.updates.adam(loss,params, learning_rate=lr) # adam most widely used update scheme

gs = nn.layers.get_all_gs(network)

gs_updates = g_updates(loss, params, gs)
updates = OrderedDict()
updates_old = nn.updates.adam(loss, params, learning_rate=lr)

stop = timeit.default_timer()
print(stop-start)

In [ ]:
size = 5
bond = math.floor(size/2)

gg_shared = theano.shared(np.array([1,2,3,4,5],dtype=np.float32))
g_params = [gg_shared[0],gg_shared[1],gg_shared[2],gg_shared[3],gg_shared[4]]
f,gamma,sigma,theta,psi = g_params
x_range = np.linspace(-bond, bond, size)
y_range = np.linspace(-bond, bond, size)

xt,yt = np.meshgrid(x_range,y_range)
xt = xt.astype(np.float32)
yt = yt.astype(np.float32)
a = gabor_filter_tensor_dev(xt,yt,g_params)
Ws = theano.tensor.stack([a,a,a,a])
Ws.reshape([4,1,5,5])
Ws.shape.eval()

In [ ]:
start = timeit.default_timer()
gabor_weight_update_dev(shape, gs[0])
stop = timeit.default_timer()
print(stop-start)

In [ ]:
start = timeit.default_timer()
## generate updates for params

for i in range (0,len(gs)):

    gs_new = gs_updates[gs[i]]
    ws = params[i*2]
    [num_filters, num_channels, filter_size, filter_size] = ws.get_value().shape
    W = gabor_weight_update_dev([num_filters, num_channels, filter_size, filter_size], gs_new)
    #updates[ws] = theano.shared(W)
    updates[ws] = W
    updates[params[i*2+1]] = updates_old[params[i*2+1]]

stop = timeit.default_timer()
print(stop-start)

In [ ]:
x = gs[1]
gs_updates[gs[1]].broadcastable
#x.broadcastable

In [2]:
from input_process import read_images
from train import run_params
import pickle
import numpy as np
import random

size = (116, 116) # resize images

pkl_file = open('../fnames_collection_2', 'r')
fnames = pkl_file.readline()
fnames = fnames.split(' ')
for i in range (0,len(fnames)):
	fnames[i] = str(fnames[i])
	fnames[i] = 'image' + fnames[i] + '.png'

'''
index = fnames[1:4000]
random.shuffle(index)
train_index = index[0:4000]
test_index = fnames[4000:4200]
'''
train_index = fnames[0:5]
test_index = fnames[10:15]



label_path = '../NewPNGlabeled/'
ori_path = '../JpegOriginalImg/'

train_input, train_label = read_images(label_path, ori_path, train_index, size)
test_input, test_label = read_images(label_path, ori_path, test_index, size)

In [3]:
epoch = 1
shape = [5,1,116,116]


input_var = train_input
label_var = train_label

[network, loss, test_loss, test_acc, output_det] = unet.network(input_var, label_var, shape)


params = nn.layers.get_all_params(network)
lr = theano.shared(nn.utils.floatX(1e-4)) # learning rate
updates = nn.updates.adam(loss,params, learning_rate=lr) # adam most widely used update scheme

gs = nn.layers.get_all_gs(network)

gs_updates = g_updates(loss, params, gs)
updates = OrderedDict()
updates_old = nn.updates.adam(loss, params, learning_rate=lr)


[network, loss, test_loss, test_acc, output_det] = unet.network(input_var, label_var, shape)


params = nn.layers.get_all_params(network)
lr = theano.shared(nn.utils.floatX(1e-4)) # learning rate
updates = nn.updates.adam(loss,params, learning_rate=lr) # adam most widely used update scheme

gs = nn.layers.get_all_gs(network)

gs_updates = g_updates(loss, params, gs)
updates = OrderedDict()
updates_old = nn.updates.adam(loss, params, learning_rate=lr)


In [ ]:
print(len(gs))
print(gs[1].shape.eval())
gs_updates[gs[1]].shape

In [4]:
gs_gradients = []
gradients = nn.updates.get_or_compute_grads(loss, params)

w_index = 0
# First Loop
# print(w_index)

ws = params[w_index*2]
g_params = gs[w_index].get_value()
ws_gradients = gradients[w_index*2]

[num_filters, num_channels, filter_size, filter_size] = ws.get_value().shape

position = int(math.floor(filter_size/2))
ws_grad = ws_gradients[:,:,position:position+1,position]

# Second and third Loop
additions = []
for i in range (0, num_filters):
    for j in range (0, num_channels):
        g = g_params[i,j,:]
        additions.append(g_psi(0,0,g))

additions =  np.array(additions,dtype=np.float32).reshape(num_filters,num_channels,-1)
g_gradients = theano.tensor.concatenate([ws_grad*0, ws_grad*0, ws_grad*0, ws_grad*0, ws_grad], axis=2)*additions
gs_gradients.append(g_gradients)
gs_updates = nn.updates.adam_dev(gs_gradients, gs)

In [5]:
g_gradients = theano.tensor.concatenate([ws_grad, ws_grad, ws_grad, ws_grad, ws_grad], axis=2)

In [16]:
g_gradients = theano.tensor.concatenate([ws_grad, ws_grad, ws_grad, ws_grad, ws_grad], axis=2)
x = np.array([ws_grad, ws_grad, ws_grad, ws_grad, ws_grad])*np.array
x[0].eval()


array([[[  6.47848664e-10]],

       [[ -1.21376098e-09]],

       [[  1.87512436e-11]],

       [[ -8.18007262e-09]],

       [[ -5.46472156e-09]],

       [[  6.03165802e-08]],

       [[  2.80293051e-07]],

       [[ -1.37698379e-07]],

       [[  4.03921606e-13]],

       [[ -2.67996857e-11]],

       [[ -2.71048317e-09]],

       [[ -1.86933224e-08]],

       [[  4.44139503e-10]],

       [[ -3.75827796e-07]],

       [[  3.04136642e-12]],

       [[ -2.07962714e-08]],

       [[  5.57484718e-11]],

       [[ -1.18298729e-07]],

       [[  1.89803240e-09]],

       [[  0.00000000e+00]],

       [[ -2.03301376e-10]],

       [[ -9.29535560e-10]],

       [[ -1.97853089e-09]],

       [[  1.10570681e-08]],

       [[  6.06412476e-10]],

       [[  1.23545230e-10]],

       [[  9.30858309e-14]],

       [[  1.50839953e-12]],

       [[ -8.92339697e-20]],

       [[ -4.00507929e-08]],

       [[ -3.59120178e-10]],

       [[ -9.92400189e-08]],

       [[  1.63540988e-14]],

       [[ 

In [6]:
g_gradients.shape.eval()

array([64,  1,  5])

In [ ]:
from collections import OrderedDict
updates = OrderedDict()
updates_old = nn.updates.adam(loss, params, learning_rate=lr)
for i in range (0,len(gs)):
    print(i)
    gs_new = gs_updates[gs[0]]
    ws = params[i*2]
    [num_filters, num_channels, filter_size, filter_size] = ws.get_value().shape
    W = gabor_weight_update([num_filters, num_channels, filter_size, filter_size], gs_new)
    #updates[ws] = theano.shared(W)
    updates[ws] = W
    updates[params[i*2+1]] = updates_old[params[i*2+1]]


In [ ]:
gradients = nn.updates.get_or_compute_grads(loss, params)
ws_gradients = gradients[0]
ws_gradients[0,0,1,1]
theano.shared(ws_gradients)

In [ ]:
def gabor_filter_dev2(x,y,params):
    f,gamma,sigma,theta,psi = params
    xt = x*theano.tensor.cos(theta) + y*theano.tensor.sin(theta)
    yt = -x*theano.tensor.sin(theta) + y*theano.tensor.cos(theta)
    z1 = -(xt**2 + (gamma*yt)**2)/(2*sigma**2)
    z2 = 2*math.pi*f*xt+psi
    #z2 = 1j*2*math.pi*f*xt+psi
    value = f**2/(math.pi*gamma)*theano.tensor.exp(z1)*theano.tensor.cos(z2)
    return value

def gabor_filter_1(x,y,params):
    f,gamma,sigma,theta,psi = params
    xt = x*math.cos(theta) + y*math.sin(theta)
    yt = -x*math.sin(theta) + y*math.cos(theta)
    z1 = -(xt**2 + (gamma*yt)**2)/(2*sigma**2)
    z2 = 1j*2*math.pi*f*xt+psi*1j
    value = f**2/(math.pi*gamma)*math.exp(z1)*cmath.exp(z2)
    return value

def gabor_filter_2(x,y,params):
    f,gamma,sigma,theta,psi = params
    xt = x*math.cos(theta) + y*math.sin(theta)
    yt = -x*math.sin(theta) + y*math.cos(theta)
    z1 = -(xt**2 + (gamma*yt)**2)/(2*sigma**2)
    z2 = 2*math.pi*f*xt+psi
    value = f**2/(math.pi*gamma)*math.exp(z1)*math.cos(z2)
    return value

In [ ]:
g_try = gs_updates[gs[0]]
updates
g_try.dtype

In [ ]:
updates_old = nn.updates.adam(loss, params, learning_rate=lr)
updates_old[params[0]].dtype

In [ ]:
gvalue = gs[0].get_value()
g = gvalue[0,0,:]
g_psi(0,0,g)
